<a href="https://colab.research.google.com/github/kcwanglucky/filetransfer/blob/master/0304_bert_ivr_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Set up

In [0]:
!pip install transformers

     |████████████████████████████████| 501kB 5.1MB/s 
     |████████████████████████████████| 870kB 69.3MB/s 
     |████████████████████████████████| 3.7MB 42.7MB/s 
     |████████████████████████████████| 1.0MB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=29ce542a85ef16fb6c781b8ab7b320d7cec3425fd95488709a1a3ff781e1220d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

In [0]:
import torch
from IPython.display import clear_output
import pandas as pd
import numpy as np
import os

### Read mapping (15 groups version)
根據客服的分類系統將小i的資料分成相對應的15類，並讀入

In [0]:
mapping_15 = pd.read_excel("0303分類對照.xlsx")

In [0]:
mapping_15.head()

,小i_index,小i_intent,ivr_index,Note
0,1,H239_How_KnowAffordHouse,2-4-1,NaN
1,2,H146_If_Apply_DecreaseRate,2-4-1,NaN
2,3,F46_Why_BuyForeignCur,2-0,NaN
3,4,C條碼,1-1-1,NaN
4,5,H161_If_TransferLoan_UnpaidLoan,2-4-1,NaN


In [0]:
idx_list = mapping_15["ivr_index"].value_counts().index

In [0]:
#將ivr的分類從0標起
ivr2contclass = {idx: i for i, idx in enumerate(idx_list)}
print(ivr2contclass)

{'2-4-1': 0, '2-0': 1, '1-4': 2, '2-4-2': 3, '1-1-1': 4, '1-2-1': 5, '1-3-1': 6, '1-1-5': 7, '3-1': 8, '1-5-2': 9, '1-3-5': 10, '1-5-1': 11, '1-3-3': 12, '2-1-8': 13, '1-1-2': 14}


In [0]:
#讀取原始intent.csv資料
df_data = pd.read_csv("intent.csv")
# 只留取index及question coluumn
df_data = df_data.iloc[:, [0, 2]]

In [0]:
i = mapping_15.loc[:, ['小i_index']].to_numpy().squeeze()
ivr = mapping_15.loc[:, ['ivr_index']].to_numpy().squeeze()

# i2ivr存取一個從小i類別對應到ivr類別的dictionary
i2ivr = {i[idx]: ivr[idx] for idx in range(len(i))}

In [0]:
# 若小i的第x類在ivr分類中有相對應的分類就回傳此分類
# 否則回傳0
def geti2ivr(x):
    if x in i2ivr:
        return i2ivr[x]
    else:
        return 0

In [0]:
df_newclass["index"] = df_data["index"].apply(geti2ivr)

In [0]:
df_newclass.to_csv("0303_intent_newclass/ivrclass.csv", index = False)
#df_newclass.read_csv("0303_intent_newclass/ivrclass.csv")

In [0]:
# 只留下有對照分類的data
df_newclass = df_newclass[df_newclass['index'] != 0 ]

In [0]:
# 將ivr分類變成連續的
def getivr2contclass(x):
    return ivr2contclass[x]

In [0]:
df_newclass["index"] = df_newclass["index"].apply(getivr2contclass)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_newclass.to_csv("0303_intent_newclass/ivrclasswithmapping.csv", index = False)

In [0]:
print("能被分到15類的資料共有　", len(df_newclass["index"]), "　筆" )

能被分到15類的資料共有　 2338 　筆


### Read extra mapping (+8 groups)

In [0]:
# sheet 1包含無法被歸類在原本ivr分類的資料，再額外將這些資料分成８個小組
mapping_extra = pd.read_excel("0303分類對照.xlsx", sheet_name = 1 )

In [0]:
m_15 = mapping_15.loc[:, ['小i_index', 'ivr_index']]
m_15 = m_15.rename(columns={"ivr_index": "大分類"})

In [0]:
m_15

,小i_index,大分類
0,1,2-4-1
1,2,2-4-1
2,3,2-0
3,4,1-1-1
4,5,2-4-1
...,...,...
248,502,2-4-1
249,503,2-4-1
250,504,2-4-2
251,506,2-4-1


In [0]:
mapping_extra

,小i_index,小i_intent,Note
0,0,H225_What_SandHouse,海砂屋
1,11,F61_If_CancelGlobalPassTranx,NaN
2,12,G06_GoodWords,NaN
3,14,H250_How_KnowBuyExpensive,NaN
4,15,C242_A_Howtoverifycard,NaN
...,...,...,...
241,494,H222_What_Attention_HouseSelfLive,NaN
242,496,G10_Roughly,NaN
243,500,F171_HowWithdrawForeignCash,外幣提款
244,501,H218_What_ProCon_BuyNewHouse,NaN


In [0]:
m_8 = mapping_extra.loc[:, ['小i_index', '大分類']]

KeyError: ignored

In [0]:
# 把原先的15類跟額外分的8類合起來
m_23 = pd.concat([m_15, m_8])

NameError: ignored

In [0]:
idx_list = m_23["大分類"].value_counts().index

In [0]:
ivr2contclass = {idx: i for i, idx in enumerate(idx_list)}
print("將新的23種ivr分類指定連續標籤：")
print(ivr2contclass)

將新的23種ivr分類指定連續標籤：
{'house': 0, '2-4-1': 1, '2-0': 2, '1-4': 3, 'card': 4, 'paypal': 5, 'miscellaneous': 6, 'cheque': 7, '2-4-2': 8, 'trash': 9, '1-1-1': 10, '1-2-1': 11, '1-3-1': 12, '1-1-5': 13, 'info': 14, 'foreign': 15, '2-1-8': 16, '3-1': 17, '1-5-2': 18, '1-3-3': 19, '1-5-1': 20, '1-3-5': 21, '1-1-2': 22}


In [0]:
m_23.dropna(axis=0, how='any', inplace=True)
m_23.head()

,小i_index,大分類
0,1,2-4-1
1,2,2-4-1
2,3,2-0
3,4,1-1-1
4,5,2-4-1


In [0]:
df = pd.read_csv("intent.csv")
# 只留取index及question coluumn
df_data = df.iloc[:, [0, 2]]
df_class = df.iloc[:, [0, 1]]

In [0]:
i = m_23.loc[:, ['小i_index']].to_numpy().squeeze()
ivr = m_23.loc[:, ['大分類']].to_numpy().squeeze()
i2ivr = {i[idx]: ivr[idx] for idx in range(len(i))}

In [0]:
print("小i分類對應到新的23種ivr分類的對照表：")
print(i2ivr)

小i分類對應到新的23種ivr分類的對照表：
{1: '2-4-1', 2: '2-4-1', 3: '2-0', 4: '1-1-1', 5: '2-4-1', 6: '1-3-1', 7: '2-4-2', 8: '2-4-1', 9: '2-0', 10: '1-4', 13: '1-4', 16: '2-4-1', 17: '2-4-1', 19: '1-2-1', 20: '2-4-1', 21: '2-0', 22: '1-4', 23: '2-4-1', 25: '2-0', 27: '2-4-1', 30: '2-4-1', 31: '2-4-1', 33: '2-0', 34: '2-4-1', 35: '1-4', 36: '2-4-1', 37: '2-4-1', 39: '2-4-1', 40: '1-3-5', 41: '1-3-3', 43: '2-4-1', 44: '1-2-1', 49: '2-4-1', 50: '1-4', 55: '2-4-1', 57: '2-0', 59: '3-1', 64: '2-4-1', 73: '2-0', 79: '2-0', 80: '2-0', 82: '1-1-2', 83: '2-4-1', 85: '2-0', 88: '1-4', 89: '2-0', 90: '2-4-1', 91: '2-0', 92: '2-0', 94: '2-0', 95: '2-1-8', 96: '2-0', 97: '2-4-1', 98: '2-4-1', 100: '1-4', 101: '2-0', 102: '1-4', 104: '2-4-1', 105: '2-4-1', 107: '1-4', 109: '2-4-1', 113: '2-4-1', 114: '2-4-1', 117: '2-0', 118: '2-0', 119: '2-0', 120: '2-0', 121: '2-4-1', 122: '2-4-1', 124: '2-4-1', 127: '2-4-1', 129: '1-1-1', 130: '1-4', 131: '2-0', 132: '2-0', 133: '2-0', 134: '2-0', 135: '2-4-2', 137: '2-4-1', 139

In [0]:
# 若小i的第x類在ivr分類中有相對應的分類就回傳此分類
# 否則回傳0
def geti2ivr(x):
    if x in i2ivr:
        return i2ivr[x]
    else:
        return 0

In [0]:
df_newclass["index"] = df_data["index"].apply(geti2ivr)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
# 只留下有對照分類的data
df_new = df_newclass[df_newclass['index'] != 0 ]

In [0]:
# 將ivr分類變成連續的
def getivr2contclass(x):
    return ivr2contclass[x]

In [0]:
df_new.dropna(axis=0, how='any', inplace=True)
df_new["index"] = df_new["index"].apply(getivr2contclass)

### 生成對照表
（原本小i的499種分類對應到新的23種ivr分類）

In [0]:
# ivr分類編碼與文字對應
ivr2ivrfull = {'1-1-1': '使用網路查詢帳單額度，繳款，補寄電子帳單，申請自動扣繳',
 '1-1-2': '語音查詢及調整可用額度',
 '1-1-5': '繳款方式',
 '1-2-1': '使用網路查詢辦卡進度，開卡',
 '1-3-1': '卡友貸款專人服務',
 '1-3-3': '網路申辦預借現金',
 '1-3-5': '合作廠師付費分期',
 '1-4': '紅利兌換，機場停車，貴賓室，旅遊保險，市區停車優惠',
 '1-5-1': '查詢優惠內容',
 '1-5-2': '網路預約機場接送',
 '2-0': '外匯及其他服務',
 '2-1-8': '支存照會',
 '2-4-1': '貸款申請（轉接專人）',
 '2-4-2': '查詢貸款當期應繳款日期及金額',
 '3-1': '使用智能小玉查詢',
 'card': 'Other-卡相關',
 'cheque': 'Other-旅行支票相關',
 'foreign': 'Other-國外旅遊相關',
 'house': 'Other-房屋相關',
 'info': 'Other-玉山歷史',
 'miscellaneous': 'Other-其他雜事（非幹話）',
 'paypal': 'Other-Paypal相關',
 'trash': 'Other-幹話'}

In [0]:
i_class = []
ivr_class = []
i_intent = []
ivr_intent = []

for k in sorted(i2ivr):
    i_class.append(k)
    ivr_class.append(i2ivr[k])
    i_intent.append(df_class[df_class['index'] == k]['intent'].iloc[0])
    ivr_intent.append(ivr2ivrfull[i2ivr[k]])
    #print(k, i2ivr[k], df_class[df_class['index'] == k]['intent'].iloc[0])

In [0]:
df_mapping = pd.DataFrame(list(zip(i_class, i_intent, ivr_class, ivr_intent)), columns=['小i_intent', '小i_full_intent', 'ivr_intent', 'ivr_full_intent'])

In [0]:
df_mapping.to_csv("mapping_detail.csv")

#### df_mapping是小i與ivr分類的對照表 <br>
ivr_intent欄位：有編碼的是按照客服分類邏輯分類的結果；若為英文字(ex: house, cheque)，則是將無法分進原本客服分類的資料，依照概略邏輯再加以分類。最後共有23類

In [0]:
df_mapping

,小i_intent,小i_full_intent,ivr_intent,ivr_full_intent
0,0,H225_What_SandHouse,house,Other-房屋相關
1,1,H239_How_KnowAffordHouse,2-4-1,貸款申請（轉接專人）
2,2,H146_If_Apply_DecreaseRate,2-4-1,貸款申請（轉接專人）
3,3,F46_Why_BuyForeignCur,2-0,外匯及其他服務
4,4,C條碼,1-1-1,使用網路查詢帳單額度，繳款，補寄電子帳單，申請自動扣繳
...,...,...,...,...
472,503,H147_What_Factor_AssessLoan,2-4-1,貸款申請（轉接專人）
473,504,H133_How_InquiryRemainAmtRate,2-4-2,查詢貸款當期應繳款日期及金額
474,505,H139_What_Diff_FixVariableInterest,house,Other-房屋相關
475,506,H81_LoanAssess_NoResult,2-4-1,貸款申請（轉接專人）


### Create split based on percentage (Get every index sampled)
邏輯：每種class都取perct %的資料進去training set

In [0]:
NUM_LABELS = len(df_new['index'].value_counts())
print("label的數量：{}".format(NUM_LABELS))

label的數量：23


In [0]:
""" 從各類別random sample出fraction比例的資料集
    data: df data that includes the "index" and "question" column
    fraction: the fraction of data you want to sample (ex: 0.7)
""" 
def bootstrap(data, fraction):
    # This function will be applied on each group of instances of the same
    # class in data.
    def sampleClass(classgroup):
        return classgroup.sample(frac = fraction)

    samples = data.groupby('index').apply(sampleClass)
    
    # If you want an index which is equal to the row in `data` where the sample came from
    # If you don't change it then you'll have a multiindex with level 0
    # being the class and level 1 being the row in `data` where
    # the sample came from.
    samples.index = samples.index.get_level_values(1)
    return samples

### 輸出預處理解果(train and test set)

In [0]:
""" 將原本全部的cleaned data依照指定的比例分成train/val/test set，
    並output成tsv檔到環境中(檔名ex: 70%train.tsv)
    df: df data that includes the "index" and "question" column
    fraction: fraction of all data to be assigned to training set
    The remaining (1-fraction) data will be equally splitted between
    validation and testing set
"""

def output_split(df, fraction = 0.7):
    df_train = bootstrap(df, fraction)
    df_remain = pd.concat([df_train, df]).drop_duplicates(keep=False)
    df_val = df_remain.sample(frac = 0.5)
    df_test = pd.concat([df_val, df_remain]).drop_duplicates(keep=False)

    # 放入label資料夾，以區分出之後unlabel的資料
    path = os.path.join("0303_intent_newclass", "data23")
    if not os.path.exists(path):
        os.makedirs(path)

    print("訓練樣本數：", len(df_train))
    df_train.to_csv(os.path.join(path, str(int(fraction * 100))+"%train.tsv"), index=False)

    print("validation樣本數：", len(df_val))
    df_val.to_csv(os.path.join(path, str(int(fraction * 100))+"%val.tsv"), index=False)

    print("預測樣本數：", len(df_test))
    df_test.to_csv(os.path.join(path, str(int(fraction * 100))+"%test.tsv"), index=False)

In [0]:
output_split(df_new, 0.7)

訓練樣本數： 2985
validation樣本數： 636
預測樣本數： 635


### 讀取前面預處理結果

In [0]:
path = os.path.join("0303_intent_newclass", "data23")
fraction = 0.7
train_path = os.path.join(path, str(int(fraction * 100))+"%train.tsv")
val_path = os.path.join(path, str(int(fraction * 100))+"%val.tsv")
test_path = os.path.join(path, str(int(fraction * 100))+"%test.tsv")
df_train = pd.read_csv(train_path).fillna("")
df_val = pd.read_csv(val_path).fillna("")
df_test = pd.read_csv(test_path).fillna("")

### 用OnlineQueryDataset來存取資料

In [0]:
"""
    實作一個可以用來讀取訓練 / 測試集的 Dataset，此 Dataset 每次將 tsv 裡的一筆成對句子
    轉換成 BERT 相容的格式，並回傳 3 個 tensors：
    - tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
    - segments_tensor：可以用來識別兩個句子界限的 binary tensor
    - label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
   
class OnlineQueryDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    # mode: in ["train", "test", "val"]
    # tokenizer: one of bert tokenizer
    # perc: percentage of data to put in training set
    # path: if given, then read data from the path(ex training set)
    def __init__(self, mode, tokenizer, perc = 70, path = None):
        assert mode in ["train", "val", "test"]  # 一般訓練你會需要 dev set
        self.mode = mode
        if not path: 
            path = os.path.join("0303_intent_newclass", "data23")
            path = os.path.join(path, str(perc) + "%" + mode + ".tsv")
        self.df = pd.read_csv(path).fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer 
    
    # 定義回傳一筆訓練 / 測試數據的函式
    #@pysnooper.snoop()  # 加入以了解所有轉換過程
    def __getitem__(self, idx):
        if self.mode == "test":
            text = self.df.iloc[idx, 1]
            label_tensor = None
        elif self.mode == "val":
            label, text = self.df.iloc[idx, :].values
            label_tensor = torch.tensor(label)
        else:
            label, text = self.df.iloc[idx, :].values
            # 將label文字也轉換成索引方便轉換成 tensor
            label_tensor = torch.tensor(label)
            
        """
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 第二個句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        """
        # 建立句子的 BERT tokens 
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([1] * len_a, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len

In [0]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()

In [0]:
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = OnlineQueryDataset("train", tokenizer=tokenizer)

In [0]:
valset = OnlineQueryDataset("val", tokenizer=tokenizer)
testset = OnlineQueryDataset("test", tokenizer=tokenizer)

In [0]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `OnlineQueryDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [0]:
# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True,  
                         collate_fn=create_mini_batch)

In [0]:
valloader = DataLoader(valset, batch_size=BATCH_SIZE,  
                         collate_fn=create_mini_batch)

### Training step
初始化一個BertForSequenceClassification Model



In [0]:
PRETRAINED_MODEL_NAME = "bert-base-chinese"
model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)
clear_output()

In [0]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

In [0]:
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
pred, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.031825795644891124


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 20
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, EPOCHS))
    print('Training...')

    # 訓練模式
    model.train()

    for data in trainloader: # trainloader is an iterator over each batch
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    logit, acc = get_predictions(model, trainloader, compute_acc=True)

    print('loss: %.3f, acc: %.3f' %
          (running_loss, acc))    
    print("")
    print("Running Validation...")

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for data in valloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]


    _, acc = get_predictions(model, valloader, compute_acc=True)
        # Move logits and labels to CPU
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        #eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        #nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(acc))

device: cuda:0

======== Epoch 1 / 50 ========
Training...
loss: 109.372, acc: 0.663

Running Validation...
  Accuracy: 0.68

======== Epoch 2 / 50 ========
Training...
loss: 61.972, acc: 0.771

Running Validation...
  Accuracy: 0.77

======== Epoch 3 / 50 ========
Training...
loss: 43.051, acc: 0.832

Running Validation...
  Accuracy: 0.82

======== Epoch 4 / 50 ========
Training...
loss: 31.905, acc: 0.868

Running Validation...
  Accuracy: 0.86

======== Epoch 5 / 50 ========
Training...
loss: 24.140, acc: 0.917

Running Validation...
  Accuracy: 0.89

======== Epoch 6 / 50 ========
Training...
loss: 18.093, acc: 0.942

Running Validation...
  Accuracy: 0.90

======== Epoch 7 / 50 ========
Training...
loss: 13.495, acc: 0.957

Running Validation...
  Accuracy: 0.92

======== Epoch 8 / 50 ========
Training...
loss: 10.479, acc: 0.959

Running Validation...
  Accuracy: 0.91

======== Epoch 9 / 50 ========
Training...
loss: 8.367, acc: 0.979

Running Validation...
  Accuracy: 0.92

===

KeyboardInterrupt: ignored

### Save Model

In [0]:
import os
output_dir = os.path.join("0303_intent_newclass", "model23", "25e70%labeled")

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to 0303_intent_newclass/model23/25e70%labeled


('0303_intent_newclass/model23/25e70%labeled/vocab.txt',
 '0303_intent_newclass/model23/25e70%labeled/special_tokens_map.json',
 '0303_intent_newclass/model23/25e70%labeled/added_tokens.json')

### Testing

In [0]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
#testset = OnlineQueryDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=32, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions= get_predictions(model, testloader)

In [0]:
def accuracy(label, pred):
    return (label == pred).sum().item()/len(label)

In [0]:
test_label = torch.cuda.FloatTensor(testset.df['index'])

In [0]:
print("Testset accuracy: %f" % accuracy(test_label, predictions))

Testset accuracy: 0.930709


若只用能依照ivr分類邏輯來分類的資料的話（共15類），可達test accuracy 98% <br>
加入其他無法確切分類後的資料，並再將之概略分為8類後，test accuracy下降至93%左右
